# 0. Load Data

In [1]:
import math 
import collections
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime

# Isolation Forest
from sklearn.model_selection import train_test_split
from sklearn.ensemble import IsolationForest
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# VAE
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from keras.layers import Input, Dense, Lambda, Flatten, Reshape
from keras.models import Model
from keras import backend as K
from keras import metrics

In [2]:
cell_ids = pd.read_csv("data/cells_reduced/cell_ids.csv",header=None)
cell_ids = [x for lst in cell_ids.values.tolist() for x in lst]
cell_ids

[2.2265366483183206e+17,
 7.315874467898523e+16,
 9.10621795573706e+16,
 3.8447326973958944e+17,
 4.262775046883192e+17]

In [21]:
cells = []
for cell_id in cell_ids:
    file_name = "cell_"+str(cell_id)+'.csv'
    cells.append(pd.read_csv(f"data/cells_reduced/{file_name}"))

In [22]:
cells[0].head()

,index,cell_id,DL_TRAFFIC_VOLUME,UL_TRAFFIC_VOLUME,Inter_X2_based_HO_prep,VoLTE_total_traffic,INTRA_FREQ_HO_SR_RATIO,RRC_SR_RATIO,CELL_AVAILABILITY_RATIO,RACH_Stp_Completion_SR_RATIO,E_RAB_QCI1_DR_RATIO,DCR_LTE_RATIO,CSSR_LTE_RATIO,LTE_INTER_ENODEB_HOSR_RATIO,Inter_RAT_HO_SR_GERAN_SRVCC_RATIO,HOUR
0,2021-05-09 00:00:00,2.226537e+17,3.779737e+10,3.947172e+09,15.0,4727.0,0.809859,0.992427,1.0,0.962688,0.000000,0.001761,0.996041,0.400000,0.963636,0
1,2021-05-09 01:00:00,2.226537e+17,3.684898e+10,4.088752e+09,6.0,3076.0,0.886792,0.993288,1.0,0.973207,0.000000,0.002468,0.995465,0.500000,1.000000,1
2,2021-05-09 02:00:00,2.226537e+17,3.292677e+10,5.016897e+09,8.0,3501.0,0.938356,0.994664,1.0,0.966330,0.013889,0.003077,0.996044,0.375000,1.000000,2
3,2021-05-09 03:00:00,2.226537e+17,3.021547e+10,5.139107e+09,9.0,2275.0,0.860215,0.994819,1.0,0.943216,0.000000,0.001721,0.995920,0.777778,0.947368,3
4,2021-05-09 04:00:00,2.226537e+17,3.082176e+10,4.250716e+09,17.0,2178.0,0.840426,0.995952,1.0,0.936256,0.000000,0.002213,0.995628,0.764706,1.000000,4


In [4]:
df_cell = cells[0].drop(['cell_id','index'],axis=1)

# 1. Isolation Forest

In [5]:
IF=IsolationForest(n_estimators=150, 
                      max_samples ='auto', 
                      max_features=1)

In [6]:
IF.fit(df_cell)
# score_samples = - score  
IF_score = -1 * IF.score_samples(df_cell)
IF_score[:20]

array([0.48337733, 0.47235561, 0.52170271, 0.50062904, 0.4644712 ,
       0.48547781, 0.49705167, 0.44710248, 0.44232739, 0.46360045,
       0.45129292, 0.45863403, 0.47047797, 0.47552665, 0.47199786,
       0.44972892, 0.44357685, 0.44221501, 0.43460012, 0.44040048])

In [ ]:
df_cell['IF_score'] = IF_score
df_cell['IF_score'].describe()

# 2. VAE

[keras - Variational AutoEncoder 官方代码范例(复杂)](https://keras.io/examples/generative/vae/)

[keras - AE/VAE 相对简单的代码构建](https://blog.keras.io/building-autoencoders-in-keras.html)

[keras - 中等程度AVE 但是有点乱](https://github.com/keras-team/keras/blob/2c8d1d03599cc03243bce8f07ed9c4a3d5f384f9/examples/variational_autoencoder.py)







[中文VAE理论+代码](https://blog.csdn.net/weixin_37737254/article/details/102920263)

[keras - 函数式API](https://keras.io/guides/functional_api/)

[keras - Conv1D](https://keras-zh.readthedocs.io/layers/convolutional/)


## 2.1 Build model

In [7]:
################  Standarlize & Split data  #################
df_cell = cells[0].drop(['cell_id','index'],axis=1)
x_train, x_test = train_test_split(df_cell)

# Standarlized seperately to avoid data leak
x_train = StandardScaler().fit_transform(x_train)
x_test  = StandardScaler().fit_transform(x_test)
x_train = np.expand_dims(x_train,axis=1)
x_test  = np.expand_dims(x_test,axis=1)

In [8]:
################ Define sampling function #################
def sampling(args):
    z_mean, z_log_sigma = args
    epsilon = K.random_normal(shape=(K.shape(z_mean)[0], latent_dim),
                              mean=0., stddev=0.1)
    return z_mean + K.exp(z_log_sigma) * epsilon

In [10]:
################ Build Encoder #################
original_dim = df_cell.shape[1]
intermediate_dim = 32
latent_dim = 2

# encoder input
inputs = keras.Input(shape=(1,original_dim))
x = layers.Conv1D(filters=intermediate_dim, kernel_size=3, padding="same", activation="relu")(inputs)
x = layers.Dropout(rate=0.2)(x)
x = layers.Flatten()(x)

# the output of encoder
z_mean      = layers.Dense(latent_dim, name="z_mean")(x)
z_log_sigma = layers.Dense(latent_dim, name="z_log_var")(x)
z           = layers.Lambda(sampling)([z_mean, z_log_sigma])

# Create encoder
encoder = keras.Model(inputs, [z_mean, z_log_sigma, z], name='encoder')
encoder.summary()

Model: "encoder"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 1, 14)]      0           []                               
                                                                                                  
 conv1d (Conv1D)                (None, 1, 32)        1376        ['input_1[0][0]']                
                                                                                                  
 dropout (Dropout)              (None, 1, 32)        0           ['conv1d[0][0]']                 
                                                                                                  
 flatten (Flatten)              (None, 32)           0           ['dropout[0][0]']                
                                                                                            

In [11]:
################ Build Decoder #################

# Create decoder
latent_inputs = keras.Input(shape=(latent_dim,), name='z_sampling')
x = layers.Dense(1*intermediate_dim, activation='relu')(latent_inputs)
x = layers.Reshape((1,intermediate_dim))(x)
x = layers.Conv1DTranspose(filters=intermediate_dim, kernel_size=3, padding="same", activation="relu")(x)
x = layers.Dropout(rate=0.2)(x)
outputs = layers.Conv1DTranspose(filters=original_dim, kernel_size=3, padding="same", activation='sigmoid')(x)
decoder = keras.Model(latent_inputs, outputs, name='decoder')
decoder.summary()

Model: "decoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 z_sampling (InputLayer)     [(None, 2)]               0         
                                                                 
 dense (Dense)               (None, 32)                96        
                                                                 
 reshape (Reshape)           (None, 1, 32)             0         
                                                                 
 conv1d_transpose (Conv1DTra  (None, 1, 32)            3104      
 nspose)                                                         
                                                                 
 dropout_1 (Dropout)         (None, 1, 32)             0         
                                                                 
 conv1d_transpose_1 (Conv1DT  (None, 1, 14)            1358      
 ranspose)                                                 

In [12]:
################ Instantiate VAE model ################# 
outputs = decoder(encoder(inputs)[2])
vae = keras.Model(inputs, outputs, name='vae_mlp')

################     Loss Function      ################# 
reconstruction_loss = keras.losses.mean_squared_error(inputs, outputs)
# reconstruction_loss *= original_dim
kl_loss = 1 + z_log_sigma - K.square(z_mean) - K.exp(z_log_sigma)
kl_loss = K.sum(kl_loss, axis=-1)
kl_loss *= -0.5
vae_loss = K.mean(reconstruction_loss + 0.0001*kl_loss) #调整比例？？
vae.add_loss(vae_loss)
vae.compile(optimizer='adam')


ps: weight of kl_loss?    

ref1.[theoretical insight](https://stats.stackexchange.com/questions/332179/how-to-weight-kld-loss-vs-reconstruction-loss-in-variational-auto-encoder)

ref2.[a more specific example](https://stats.stackexchange.com/questions/341954/balancing-reconstruction-vs-kl-loss-variational-autoencoder)

In [13]:
# Train model
vae.fit(x_train, x_train,
        epochs=100,
        batch_size=32,
        validation_data=(x_test, x_test))

Epoch 1/100
39/39 [==============================] - 1s 9ms/step - loss: 1.2259 - val_loss: 1.1834
Epoch 2/100
39/39 [==============================] - 0s 3ms/step - loss: 1.0984 - val_loss: 1.0088
Epoch 3/100
39/39 [==============================] - 0s 2ms/step - loss: 0.9944 - val_loss: 0.9783
Epoch 4/100
39/39 [==============================] - 0s 3ms/step - loss: 0.9618 - val_loss: 0.9419
Epoch 5/100
39/39 [==============================] - 0s 3ms/step - loss: 0.9313 - val_loss: 0.9158
Epoch 6/100
39/39 [==============================] - 0s 2ms/step - loss: 0.9155 - val_loss: 0.9040
Epoch 7/100
39/39 [==============================] - 0s 2ms/step - loss: 0.9049 - val_loss: 0.8929
Epoch 8/100
39/39 [==============================] - 0s 3ms/step - loss: 0.8979 - val_loss: 0.8831
Epoch 9/100
39/39 [==============================] - 0s 3ms/step - loss: 0.8886 - val_loss: 0.8755
Epoch 10/100
39/39 [==============================] - 0s 3ms/step - loss: 0.8817 - val_loss: 0.8713
Epoch 11/

39/39 [==============================] - 0s 3ms/step - loss: 0.8201 - val_loss: 0.8182
Epoch 84/100
39/39 [==============================] - 0s 3ms/step - loss: 0.8178 - val_loss: 0.8177
Epoch 85/100
39/39 [==============================] - 0s 3ms/step - loss: 0.8208 - val_loss: 0.8162
Epoch 86/100
39/39 [==============================] - 0s 3ms/step - loss: 0.8208 - val_loss: 0.8155
Epoch 87/100
39/39 [==============================] - 0s 3ms/step - loss: 0.8215 - val_loss: 0.8160
Epoch 88/100
39/39 [==============================] - 0s 3ms/step - loss: 0.8195 - val_loss: 0.8146
Epoch 89/100
39/39 [==============================] - 0s 3ms/step - loss: 0.8219 - val_loss: 0.8148
Epoch 90/100
39/39 [==============================] - 0s 2ms/step - loss: 0.8156 - val_loss: 0.8130
Epoch 91/100
39/39 [==============================] - 0s 3ms/step - loss: 0.8183 - val_loss: 0.8115
Epoch 92/100
39/39 [==============================] - 0s 3ms/step - loss: 0.8173 - val_loss: 0.8133
Epoch 93/100


In [11]:
vae.save("model/2_VAE/cell0/")

INFO:tensorflow:Assets written to: model/2_VAE/cell0\assets


## 2.2 [Detect Anomaly](https://towardsdatascience.com/hands-on-anomaly-detection-with-variational-autoencoders-d4044672acd5)
Detect anomaly by finding those who have a high reconstruction loss.


Steps:
1. Measure error between the original train (clean/normal) set and the output of the model, and generate an error vector representing the error term of each sample.

2. Find a relatively extreme value on that vector to use as your error threshold.

3. Run the model over the test or real data, in which anomalies are probably mixed with normal data.

4. Measure the reconstruction error and mark samples that exhibit an error term higher than the error threshold as anomalies.

In [14]:
def mse(original,reconstructed):
    return np.mean(np.sum((original-reconstructed)**2,axis=1),axis=1)

In [15]:
from sklearn.preprocessing import MinMaxScaler
x_train_pred = vae.predict(x_train)
x_test_pred  = vae.predict(x_test)

x_test_loss = mse(x_test,x_test_pred)
x_test_loss[:20]

array([0.44500221, 0.25751857, 0.49622781, 1.06377065, 0.98245546,
       0.35493252, 0.26544952, 0.28748259, 0.48888827, 1.48943495,
       1.59046277, 0.3092292 , 0.61331426, 0.77651233, 0.90566509,
       1.12027929, 0.64347693, 1.09713459, 0.78433523, 0.72332917])

In [16]:
#just to show ... 
MinMaxScaler().fit_transform(np.array(x_test_loss).reshape(-1, 1))[:20]

array([[0.01272717],
       [0.00647947],
       [0.01443421],
       [0.03334698],
       [0.03063724],
       [0.00972569],
       [0.00674376],
       [0.00747799],
       [0.01418963],
       [0.0475318 ],
       [0.05089844],
       [0.00820268],
       [0.01833599],
       [0.0237744 ],
       [0.02807828],
       [0.03523007],
       [0.01934113],
       [0.0344588 ],
       [0.02403509],
       [0.02200212]])

# 3. VAE + IF

In [17]:
# standarlized data 
df_cell = StandardScaler().fit_transform(df_cell)
df_cell  = np.expand_dims(df_cell,axis=1)

encoder_output = encoder.predict(df_cell)
encoder_output

[array([[ 2.2117567, -2.6360521],
        [ 3.005916 , -2.505186 ],
        [ 1.5189962, -3.5336614],
        ...,
        [-5.9262166,  7.3983345],
        [-7.022243 ,  9.947377 ],
        [-6.2559733,  8.537143 ]], dtype=float32),
 array([[-1.9449544, -2.0011637],
        [-2.050037 , -2.2249632],
        [-2.3688183, -2.2569823],
        ...,
        [-2.668743 , -0.8112968],
        [-3.071959 , -1.2745733],
        [-3.595646 , -1.815458 ]], dtype=float32),
 array([[ 2.2124903, -2.628674 ],
        [ 3.0209   , -2.503102 ],
        [ 1.5251638, -3.5338354],
        ...,
        [-5.9281125,  7.328612 ],
        [-7.0222306,  9.924319 ],
        [-6.254571 ,  8.52362  ]], dtype=float32)]

In [18]:
def feature_from_encoder(encoder_output):
    # take mean and log_var as features
    df_encoded_mean   = pd.DataFrame(encoder_output[0],columns=["mean1","mean2"])
    df_encoded_logvar = pd.DataFrame(encoder_output[1],columns=["log_var1","log_var2"])
    df_encoded        = df_encoded_mean.join(df_encoded_logvar)
    return df_encoded

df_encoded = feature_from_encoder(encoder_output)
df_encoded

,mean1,mean2,log_var1,log_var2
0,2.211757,-2.636052,-1.944954,-2.001164
1,3.005916,-2.505186,-2.050037,-2.224963
2,1.518996,-3.533661,-2.368818,-2.256982
3,1.402188,-3.422911,-2.038118,-2.176879
4,1.999225,-1.698540,-1.908680,-2.162398
...,...,...,...,...
1650,-6.061464,8.394439,-2.240656,-0.697308
1651,-5.778695,8.521473,-2.654520,-0.971294
1652,-5.926217,7.398335,-2.668743,-0.811297
1653,-7.022243,9.947377,-3.071959,-1.274573


In [19]:
VAE_IF=IsolationForest(n_estimators=150, 
                      max_samples ='auto', 
                      max_features=1)
VAE_IF.fit(df_encoded)

IsolationForest(max_features=1, n_estimators=150)

In [20]:
VAE_IF_score = -1 * VAE_IF.score_samples(df_encoded)
VAE_IF_score[:20]

array([0.46744059, 0.48029628, 0.47513529, 0.45947107, 0.45043746,
       0.46926745, 0.62673232, 0.41720654, 0.41796033, 0.42949577,
       0.4537181 , 0.46111905, 0.43473572, 0.43110157, 0.44674073,
       0.45667155, 0.42423758, 0.4510423 , 0.42525502, 0.42354986])

# 4.Visualization

In [ ]:
def anomalyPercent(model, df_test):
    """
    Calculate the percentage of anomalies in the testset
    
    """
    df_pred = df_test.copy()
    df_pred['anomaly'] = model.predict(df_test)
    pct  = (df_pred['anomaly']==-1).sum()/len(df_test)
    
#     # map 1 -> 0, -1 -> 1
#     reset_value = lambda x: 1 if x==-1 else 0
#     df_test['anomaly'] = df_test['anomaly'].map(reset_value)
    
    return pct, df_pred

In [ ]:
def anomaly_visualization(df):
    """
    Visualize anomalies in 2D/3D scatter plot
    
    INPUT
    @df : a dataframe where the prediction result is in the 'anomaly' column
    
    """
    index = df.index
    outlier_index = list(df[df['anomaly']==-1].index)
    
    # nomalize the metrics
    X = StandardScaler().fit_transform(df.drop('anomaly',axis=1))
    
    # recude the dimension for visualization
    X_3d = pd.DataFrame(PCA(3).fit_transform(X),index=index)
    X_3d_outliers = X_3d.reindex(outlier_index)
    X_2d = pd.DataFrame(PCA(2).fit_transform(X),index=index)
    X_2d_outliers = X_2d.reindex(outlier_index)
    
    # plot
    fig = plt.figure(figsize=(10,5))
    # 3D
    ax1 = fig.add_subplot(121,  projection='3d')
    ax1.scatter(X_3d.loc[:,0],X_3d.loc[:,1],X_3d.loc[:,2],
                s=4,lw=1,label="inliers",c="green",alpha=0.5)
    ax1.scatter(X_3d_outliers.loc[:,0], X_3d_outliers.loc[:,1], X_3d_outliers.loc[:,2],
               s=60,marker="x",lw=2,label="outliers",c="red")
    ax1.legend()
    ax1.set_title("Isolation Prediction (3D)")
    # 2D
    ax2 = fig.add_subplot(133)
    ax2.scatter(X_2d.loc[:,0],X_2d.loc[:,1],c="green",
                s=5,label="inliers",alpha=0.8)
    ax2.scatter(X_2d_outliers.iloc[:,0], X_2d_outliers.iloc[:,1],
                s=60,marker="x",lw=2,c='red',label="outliers")
    ax2.legend()
    ax2.set_title("Isolation Prediction (2D)")
    plt.show()
    

In [ ]:
for i in range(len(list_cells)):
    # get the dataframe of a certain cell
    df_c = list_cells[i].drop(['cell_id','index'], axis=1)
    
    # split train set and test set 
    df_train, df_test = train_test_split(df_c)
    # train model 
    model.fit(df_train)
    pct, df_pred = anomalyPercent(model, df_test)
    print("="*75)
    print(f"cell id: {cell_ids[i]}")
    print("Percentage of outliers: %.3f "%pct)
    anomaly_visualization(df_pred)
    print("\n\n")